# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = self.get_value()

    def get_value(self):
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11  # Can be adjusted to 1 in gameplay
        else:
            return int(self.rank)

    def __repr__(self):
        return f"{self.rank}{self.suit}"

class Deck:
    suits = ['H', 'D', 'C', 'S']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

    def __init__(self, num_decks=6):
        self.num_decks = num_decks
        self.cards = self.generate_deck()
        self.shuffle()
        self.plastic_card_position = random.randint(15, len(self.cards) - 15)
    
    def generate_deck(self):
        return [Card(suit, rank) for suit in self.suits for rank in self.ranks] * self.num_decks
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def draw_card(self):
        if len(self.cards) == self.plastic_card_position:
            self.shuffle()
        return self.cards.pop() if self.cards else None

class Player:
    def __init__(self, name, chips=1000, is_human=False):
        self.name = name
        self.chips = chips
        self.hand = []
        self.is_human = is_human
    
    def place_bet(self, amount):
        if amount <= self.chips:
            self.chips -= amount
            return amount
        return 0
    
    def receive_card(self, card):
        self.hand.append(card)
    
    def show_hand(self):
        return self.hand

    def hand_value(self):
        value = sum(card.value for card in self.hand)
        num_aces = sum(1 for card in self.hand if card.rank == 'A')
        while value > 21 and num_aces:
            value -= 10
            num_aces -= 1
        return value

    def has_busted(self):
        return self.hand_value() > 21

    def should_hit(self, dealer_face_up_card):
        self.print_status()
        if self.is_human:
            while True:
                choice = input(f"{self.name}, your hand: {self.show_hand()} (Value: {self.hand_value()}). Dealer's face-up card: {dealer_face_up_card}. Hit or Stay? (h/s): ")
                if choice.lower() in ['h', 's']:
                    decision = "hit" if choice.lower() == 'h' else "stay"
                    print(f"{self.name} decided to {decision}.")
                    return choice.lower() == 'h'
        return self.hand_value() < 12
    
    def print_status(self):
        print(f"{self.name}: {self.show_hand()} (Value: {self.hand_value()}) {'BUSTED' if self.has_busted() else ''}")

class Dealer(Player):
    def __init__(self):
        super().__init__(name="Dealer")
    
    def should_hit(self):
        return self.hand_value() < 17

class BlackjackGame:
    def __init__(self, num_players=1, include_human=False):
        self.deck = Deck()
        self.dealer = Dealer()
        self.players = [Player(f"Player {i+1}") for i in range(num_players)]
        if include_human:
            self.human_player = Player("Human", is_human=True)
            self.players.append(self.human_player)
        else:
            self.human_player = None
    
    def determine_winner(self):
        dealer_value = self.dealer.hand_value()
        print("\n--- Round Results ---")
        for player in self.players:
            player_value = player.hand_value()
            if player.has_busted():
                print(f"{player.name} BUSTED! Dealer wins.")
            elif self.dealer.has_busted():
                print(f"Dealer BUSTED! {player.name} wins.")
            elif player_value > dealer_value:
                print(f"{player.name} wins with {player_value} over dealer's {dealer_value}.")
            elif player_value < dealer_value:
                print(f"Dealer wins against {player.name}. {player_value} vs {dealer_value}.")
            else:
                print(f"{player.name} and Dealer tied at {player_value}.")
        print("--------------------\n")
    
    def play_round(self):

        print("\n--- New Round ---")

        print("Placing bets...")
        for player in self.players:
            bet = player.place_bet(10)
            if player.is_human:
                print(f"What is your bet?")
                bet = int(input())
            print(f"{player.name} placed a bet of {bet}.")
        
        

        for player in self.players:
            player.receive_card(self.deck.draw_card())
            player.receive_card(self.deck.draw_card())
        self.dealer.receive_card(self.deck.draw_card())
        dealer_face_up_card = self.deck.draw_card()
        self.dealer.receive_card(dealer_face_up_card)
        
        print(f"Dealer's face-up card: {dealer_face_up_card}")

        # Print current hands
        for player in self.players:
            player.print_status()
        
        # Players place bets
        
  

# Run a playable game with a human player
game = BlackjackGame(num_players=2, include_human=True)
game.play_round()


Dealer's face-up card: 8C
Player 1: [AD, 10S] (Value: 21) 
Player 2: [10H, 6C] (Value: 16) 
Human: [2H, 4D] (Value: 6) 
